In [88]:
import datetime
from pathlib import Path
import backtrader as bt

In [89]:
class TestStrategy(bt.Strategy):

    def __init__(self):
        self.msft = self.datas[0]
        self.dataclose = self.msft.close

        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order: bt.Order):
        # called when order status changes

        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                print(
                    f"""BUY EXECUTED 
                    Price: {self.buyprice} 
                    Cost: {order.executed.value}
                    Commision: {self.buycomm}
                    """)

            elif order.issell():

                sellprice = order.executed.price
                sellcomm = order.executed.comm
                print(
                    f"""SELL EXECUTED 
                    Price: {sellprice} 
                    Cost: {order.executed.value}
                    Commision: {sellcomm}
                    """)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"Order has been: {order.status}")

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade: bt.Trade):
        if trade.isclosed:
            print(f"""TRADE WAS CLOSED: 
                GROSS: {trade.pnl},
                NET: {trade.pnlcomm}""")

    def next(self):
        current_time = self.msft.datetime.date(0)
        current_open = self.msft.open[0]
        current_close = self.dataclose[0]

        print(
            f'Open: {current_open} Close: {current_close}, at {current_time} ')

        # an order is pending
        if self.order:
            return

        if not self.position:
            # might buy
            # buy singal
            if self.dataclose[0] < self.dataclose[-1] < self.dataclose[-2]:
                print(f"WANT BUY: {current_close}")
                self.order = self.buy()
        else:
            # might sell
            # sell signal
            if len(self) >= (self.bar_executed + 5):
                print(f"WANT SELL: {current_close}")
                self.order = self.sell()

In [90]:


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100_000.0)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.addstrategy(TestStrategy)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # add data
    datapath = Path('msft_day.csv')
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        fromdate=datetime.datetime(2000, 1, 1),
        todate=datetime.datetime(2000, 12, 31),
    )

    cerebro.adddata(data)
    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Open: 36.32 Close: 36.07, at 2000-01-03 
Open: 35.14 Close: 34.85, at 2000-01-04 
Open: 34.38 Close: 35.21, at 2000-01-05 
Open: 34.71 Close: 34.04, at 2000-01-06 
Open: 33.61 Close: 34.48, at 2000-01-07 
Open: 35.1 Close: 34.73, at 2000-01-10 
Open: 34.5 Close: 33.84, at 2000-01-11 
Open: 33.57 Close: 32.74, at 2000-01-12 
WANT BUY: 32.74
BUY EXECUTED 
                    Price: 32.29 
                    Cost: 32.29
                    Commision: 0.03229
                    
Open: 32.29 Close: 33.36, at 2000-01-13 
Open: 33.16 Close: 34.73, at 2000-01-14 
Open: 34.6 Close: 35.68, at 2000-01-18 
Open: 34.19 Close: 33.11, at 2000-01-19 
Open: 33.13 Close: 32.8, at 2000-01-20 
Open: 33.11 Close: 32.1, at 2000-01-21 
WANT SELL: 32.1
SELL EXECUTED 
                    Price: 32.12 
                    Cost: 32.29
                    Commision: 0.032119999999999996
                    
TRADE WAS CLOSED: 
                GROSS: -0.1700000000000017,
      